In [ ]:
from matplotlib.colors import SymLogNorm
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates.sky_coordinate import SkyCoord
import astropy.units as u
from astropy.cosmology import Planck15 as cosmo
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725, Ob0 = 0.0486, m_nu = [0, 0, 0.06] * u.eV)
from astropy.wcs import WCS
from minot import model, model_plots
from astropy import constants as const
from astropy.wcs import WCS

# Modify plotting parameters
dict_base = {'font.size':        16,
             'legend.fontsize':  16,
             'xtick.labelsize':  16,
             'ytick.labelsize':  16,
             'axes.labelsize':   16,
             'axes.titlesize':   16,
             'figure.titlesize': 16,    
             'figure.figsize':[8.0, 6.0],
             'figure.subplot.right':0.97,
             'figure.subplot.left':0.15,
             'font.family':'serif',
             'figure.facecolor': 'white',
             'legend.frameon': True}
plt.rcParams.update(dict_base)

In [ ]:
clust = model.Cluster(name='A3558', redshift=0.048, M500=671000000000000.0*u.Msun, 
                        cosmology=cosmo, silent=False,
                        output_dir='~/A3558')

In [ ]:
#----- Some parameters that can be set at when defining the cluster
clust.coord = SkyCoord("201.97d -31.49d", frame="icrs", unit="deg")

#----- Define the relevant ICM model (available models are defined in model_modpar.py)

# Density profile of the target thermal gas
clust.density_gas_model = {'name':'beta', 'n_0':0.46e-2*u.cm**-3, 'beta':0.58, 'r_c':223*u.kpc}

# Pressure profile of the target thermal gas
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.2e-2*u.keV/u.cm**3, 'c500':3.2, 'a':1.5, 'b':3.1, 'c':0.0}

#Exact from Planck 13
clust.pressure_gas_model = {'name':'GNFW', 'P_0':(6.04*0.0030143804978998274)*u.keV/u.cm**3, 'c500':1.77, 'a':1.12, 'b':3.58, 'c':0.31}

#Gong 20
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.99e-2*u.keV/u.cm**3, 'c500':1.16, 'a':2.66, 'b':5.48, 'c':0.3}

#Plank Coma Universal test
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.57e-2*u.keV/u.cm**3, 'c500':1.17, 'a':1.05, 'b':5.49, 'c':0.3}

# Set the cosmic ray density profile to the same shape as the thermal gas density
clust.set_density_crp_isodens_scal_param()

# Cosmic ray proton spectrum
clust.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}

# Cosmic ray proton normalization
clust.X_crp_E = {'X':0.05, 'R_norm':clust.R500} # X is the CRp/thermal energy ratio, normalized within R_norm

#Set magnetic field profile
clust.set_magfield_isodens_scal_param(5*u.uG, scal=0.5)

In [ ]:
# Defines the grid
clust.map_reso = 0.01*u.deg
clust.map_fov = 5.0*u.deg

# The center of the map can be offset wrt the cluster
clust.map_coord = SkyCoord("201.97d -31.49d", frame="icrs", unit="deg")

# Get the corresponding header
header = clust.get_map_header()

In [ ]:
template = clust.get_gamma_map(Emin=None, Emax=None, 
                               Energy_density=False, 
                               Normalize=True) # The map is normalized to the total flux, to be in unit of sr-1

In [ ]:
rad500 = clust.theta500.to_value('deg')
radtru = clust.theta_truncation.to_value('deg')

fig = plt.figure(figsize=(10, 8))
ax = plt.subplot(111, projection=WCS(header))
plt.imshow(template, vmin=0, origin='lower', cmap='magma', norm=SymLogNorm(1))

# Show R500
circle500 = matplotlib.patches.Ellipse((clust.coord.ra.value, clust.coord.dec.value),
                                        2*rad500/np.cos(clust.coord.dec.value*np.pi/180), 2*rad500,
                                        linewidth=2, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
# Show R_truncation
circletru = matplotlib.patches.Ellipse((clust.coord.ra.value, clust.coord.dec.value),
                                        2*radtru/np.cos(clust.coord.dec.value*np.pi/180), 2*radtru,
                                        linewidth=1, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
ax.add_patch(circle500)
ax.add_patch(circletru)
plt.colorbar()
plt.title(clust.name+', gamma template ($sr^{-1}$)')